In [2]:
import pandas as pd
import numpy as np
import os

from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

In [3]:
source = 'source/gapdata010.xls'

In [11]:
data = pd.read_excel(source)

In [12]:
data = data.set_index('Country')

In [13]:
data = data.dropna(how='all')

In [18]:
data = data.reset_index()

In [15]:
len(data)

1744

In [ ]:
# entities

In [19]:
country = data[['Country']].copy()

In [20]:
country['country'] = country['Country'].map(to_concept_id)

In [21]:
country.columns = ['name', 'country']

In [22]:
country = country.drop_duplicates()

In [24]:
country.to_csv('../ddf--entities--country.csv', index=False)

In [25]:
# concepts

In [26]:
data.columns

Index(['Country', 'year', 'MMR', 'Live Births', 'Maternal deaths',
       'Women reproductive age (15-49)', 'MM-rate', 'Unnamed: 7',
       'Skilled attendance %', 'Source MMR, maternal death and Live birth',
       'Comment', 'Source women reproductive age', 'Comments'],
      dtype='object')

In [27]:
conc = ['MMR', 'Live Births', 'Maternal deaths',
       'Women reproductive age (15-49)', 'MM-rate']

In [28]:
conc_id = list(map(to_concept_id, conc))

In [29]:
cdf = pd.DataFrame([], columns=['concept', 'name', 'concept_type'])

In [30]:
cdf['name'] = conc
cdf['concept'] = conc_id

In [31]:
cdf['concept_type'] = 'measure'

In [32]:
desc = pd.DataFrame([['name', 'Name', 'string'], 
                     ['year', 'Year', 'string'], 
                     ['country', 'Country', 'entity_domain']], columns=cdf.columns)

In [33]:
call = pd.concat([desc, cdf])

In [35]:
call.to_csv('../ddf--concepts.csv', index=False)

In [36]:
# datapoints

In [48]:
dps_cols = ['Country', 'year', *conc]

In [49]:
dps = data[dps_cols].copy()

In [50]:
dps.columns = list(map(to_concept_id, dps.columns))

In [51]:
dps['country'] = dps['country'].map(to_concept_id)

In [52]:
dps = dps.set_index(['country', 'year'])

In [54]:
for i, df in dps.items():
    path = os.path.join('../', 'ddf--datapoints--{}--by--country--year.csv'.format(i))
    
    df_ = df.reset_index()
    
    df_.sort_values(by=['country', 'year']).dropna().to_csv(path, index=False)

In [45]:
!open .